In [46]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from stable_baselines3.common.atari_wrappers import (  
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
)

In [47]:
#CONSTANTS
total_timesteps = 1000000
num_envs = 4
num_steps = 128
num_epochs = 4
batch_size = 128
num_updates = total_timesteps // batch_size


In [57]:
def make_env(gym_id, seed, idx, capture_video, run_name):
    def env_create():
        env = gym.make(gym_id, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env = gym.wrappers.RecordVideo(env, f"videos/{run_name}", lambda episode_id: episode_id % 100 == 0)
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.unwrapped.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return env_create

In [58]:
def layer_initilization(layer, std=np.sqrt(2), bias_const=0.0):
    nn.init.orthogonal_(layer.weight, std)
    nn.init.constant_(layer.bias.data, bias_const)
    return layer

class PPOAgent(nn.Module):
    
    def __init__(self, n_actions, n_frames = 4 ):
        super(PPOAgent, self).__init__()
        
        self.conv = nn.Sequential(
            layer_initilization(nn.Conv2d(n_frames, 32, kernel_size=8, stride=4)),
            nn.ReLU(),
            layer_initilization(nn.Conv2d(32, 64, kernel_size=4, stride=2)),
            nn.ReLU(),
            layer_initilization(nn.Conv2d(64, 64, kernel_size=3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512)
        )
        
        self.actor = nn.Sequential(
            layer_initilization(nn.Linear(512, n_actions))
        )
        
        self.critic = nn.Sequential(
            layer_initilization(nn.Linear(512, 1))
        )
        
        
    def get_value(self, x):
        return self.critic(self.conv(x))
    
    def get_action(self, x, action = None):
            logits = self.actor(self.conv(x))
            dist = torch.distributions.Categorical(logits=logits)
            if action == None:
                action = dist.sample()
            value = self.critic(self.conv(x))
            return action, dist.log_prob(action), dist.entropy(), value
            

In [59]:
def rollout(envs, agent, num_steps, gamma = 0.99, gae_lambda = 0.95):
    
    
    observations = torch.zeros((num_steps , num_envs) + envs.single_observation_space.shape, dtype=torch.float32)
    actions = torch.zeros((num_steps, num_envs) + envs.single_action_space.shape, dtype=torch.int32)
    rewards = torch.zeros((num_steps, num_envs), dtype=torch.float32)
    values = torch.zeros((num_steps, num_envs), dtype=torch.float32)
    logprobs = torch.zeros((num_steps, num_envs), dtype=torch.float32)
    dones = torch.zeros((num_steps, num_envs), dtype=torch.float32)
    #truncs = torch.zeros((num_steps, num_envs), dtype=torch.float32)

    next_obs = torch.Tensor(envs.reset()[0])
    next_done = torch.zeros(num_envs)
    # next_trunc = torch.zeros(num_envs)
    
    for step in range(num_steps):
        observations[step] = next_obs
        dones[step] = torch.Tensor(next_done)
        #truncs[step] = next_trunc
        
        with torch.no_grad():
            action, logprob, _, value = agent.get_action(next_obs)
            
        actions[step] = action
        values[step] = value.view(-1)
        logprobs[step] = logprob
        
        next_obs, reward, next_done, _, _ = envs.step(action.cpu().numpy())
        next_obs = torch.Tensor(next_obs)
        rewards[step] = torch.Tensor(reward)
        
    with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            advantages = torch.zeros_like(rewards)
            lastgaelam = 0
            for t in reversed(range(num_steps)):
                if t == num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values[t + 1]
                delta = rewards[t] + gamma * nextvalues * nextnonterminal - values[t]
                advantages[t] = lastgaelam = delta + gamma * gae_lambda * nextnonterminal * lastgaelam
            returns = advantages + values
    
    return observations, actions, returns, values, advantages, logprobs, dones
        
    

In [60]:
def update_agent(agent, optimizer, observations, actions, returns, values, advantages, logprobs, clip_param = 0.2, vf_coef = 0.5, ent_coef = 0.01):
    
    observations = observations.view(-1, *observations.shape[2:])
    actions = actions.view(-1, *actions.shape[2:])
    returns = returns.view(-1)
    values = values.view(-1)
    advantages = advantages.view(-1)
    logprobs = logprobs.view(-1)
    indices = torch.randperm(observations.size(0))
    observations = observations[indices]
    actions = actions[indices]
    returns = returns[indices]
    advantages = advantages[indices]
    logprobs = logprobs[indices]
    advantages = advantages.view(-1)
    logprobs = logprobs.view(-1)
    num_batches = observations.size(0) // 128
    for _ in range(num_epochs):
        for batch in range(num_batches):
            batch_indices = slice(batch * 128, (batch + 1) * 128)
            
            obs_batch = observations[batch_indices]
            act_batch = actions[batch_indices]
            ret_batch = returns[batch_indices]
            adv_batch = advantages[batch_indices]
            norm_adv_batch = (adv_batch - adv_batch.mean()) / (adv_batch.std() + 1e-8)
            logprob_batch = logprobs[batch_indices]
            
            _, logprob, entropy, value = agent.get_action(obs_batch, act_batch)
            ratio = (logprob - logprob_batch).exp()
            policy_loss = -torch.min(ratio * norm_adv_batch, torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * norm_adv_batch).mean()
            value_loss = 0.5 * (value - ret_batch).pow(2).mean()
            entropy_loss = entropy.mean()
            loss = policy_loss + value_loss * vf_coef - entropy_loss * ent_coef
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), 0.5)
            optimizer.step()
            
    return loss.item(), policy_loss.item(), value_loss.item(), entropy_loss.item()
    
    
    

In [62]:
#FINAL TRAINING LOOP
envs = gym.vector.SyncVectorEnv([make_env("ALE/Pong-v5", 9, i, False, "Pong") for i in range(num_envs)])
agent = PPOAgent(envs.single_action_space.n)
optimizer = torch.optim.Adam(agent.parameters(), lr=2.5e-4, eps=1e-5)
for update in range(num_updates):
    observations, actions, returns, values, advantages, logprobs, dones = rollout(envs, agent, num_steps)
    loss, policy_loss, value_loss, entropy_loss = update_agent(agent, optimizer, observations, actions, returns, values, advantages, logprobs)
    print(f"Update: {update}, Loss: {loss}, Policy Loss: {policy_loss}, Value Loss: {value_loss}, Entropy Loss: {entropy_loss}")

Update: 463, Loss: 0.1704680174589157, Policy Loss: -0.055769290775060654, Value Loss: 0.4720878005027771, Entropy Loss: 0.9806581735610962
Update: 464, Loss: 0.14615431427955627, Policy Loss: -0.04496391862630844, Value Loss: 0.40233680605888367, Entropy Loss: 1.0050170421600342
Update: 465, Loss: -0.009928026236593723, Policy Loss: -0.08135127276182175, Value Loss: 0.163709357380867, Entropy Loss: 1.0431432723999023
Update: 466, Loss: 0.19607031345367432, Policy Loss: -0.03554758429527283, Value Loss: 0.48416972160339355, Entropy Loss: 1.0466958284378052
Update: 467, Loss: 0.011014502495527267, Policy Loss: -0.07798893004655838, Value Loss: 0.19915689527988434, Entropy Loss: 1.0575015544891357
Update: 468, Loss: 0.0933627113699913, Policy Loss: -0.04766623675823212, Value Loss: 0.29999932646751404, Entropy Loss: 0.8970717191696167
Update: 469, Loss: 0.1386347860097885, Policy Loss: -0.03892343118786812, Value Loss: 0.3759137690067291, Entropy Loss: 1.039867639541626
Update: 470, Loss

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-300.mp4


Update: 486, Loss: 0.010589079931378365, Policy Loss: -0.07597603648900986, Value Loss: 0.19308465719223022, Entropy Loss: 0.997721254825592
Update: 487, Loss: 0.08747320622205734, Policy Loss: -0.043640896677970886, Value Loss: 0.28317826986312866, Entropy Loss: 1.0475029945373535
Update: 488, Loss: 0.01122464518994093, Policy Loss: -0.05000896751880646, Value Loss: 0.14172163605690002, Entropy Loss: 0.962720513343811
Update: 489, Loss: 0.14002427458763123, Policy Loss: -0.04726022854447365, Value Loss: 0.3901309669017792, Entropy Loss: 0.778099000453949
Update: 490, Loss: 0.031720589846372604, Policy Loss: -0.0785830095410347, Value Loss: 0.24182459712028503, Entropy Loss: 1.0608699321746826
Update: 491, Loss: 0.035679928958415985, Policy Loss: -0.06236526742577553, Value Loss: 0.21460109949111938, Entropy Loss: 0.9255355596542358
Update: 492, Loss: 0.16065163910388947, Policy Loss: -0.06532537937164307, Value Loss: 0.4694317579269409, Entropy Loss: 0.873885452747345
Moviepy - Buildi

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-300.mp4
Update: 493, Loss: -0.00861695408821106, Policy Loss: -0.09749908745288849, Value Loss: 0.1957366168498993, Entropy Loss: 0.8986175060272217
Update: 494, Loss: 0.054064568132162094, Policy Loss: -0.06832731515169144, Value Loss: 0.26380789279937744, Entropy Loss: 0.9512062072753906
Update: 495, Loss: 0.03707762435078621, Policy Loss: -0.0625455304980278, Value Loss: 0.21971380710601807, Entropy Loss: 1.0233749151229858
Update: 496, Loss: 0.042475830763578415, Policy Loss: -0.053720925003290176, Value Loss: 0.21110792458057404, Entropy Loss: 0.9357208013534546
Moviepy - Building video /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-300.mp4.
Moviepy - Writing video /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-300.mp4



Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-300.mp4


...

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-1400.mp4
Update: 5180, Loss: 0.042447663843631744, Policy Loss: -0.031326938420534134, Value Loss: 0.16371522843837738, Entropy Loss: 0.8083013892173767
Update: 5181, Loss: 0.0215986967086792, Policy Loss: -0.056581877171993256, Value Loss: 0.1741989701986313, Entropy Loss: 0.8918910622596741
Update: 5182, Loss: 0.060776859521865845, Policy Loss: -0.04732649400830269, Value Loss: 0.232987642288208, Entropy Loss: 0.8390473127365112
Update: 5183, Loss: 0.03252986818552017, Policy Loss: -0.033629823476076126, Value Loss: 0.1516207456588745, Entropy Loss: 0.9650683403015137
Update: 5184, Loss: 0.05070209503173828, Policy Loss: -0.03366190940141678, Value Loss: 0.18333713710308075, Entropy Loss: 0.7304562926292419
Update: 5185, Loss: -0.009888975881040096, Policy Loss: -0.07936571538448334, Value Loss: 0.15689650177955627, Entropy Loss: 0.8971511125564575
Update: 5186, Loss: 0.00597023

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-1500.mp4
Update: 5563, Loss: 0.04626983776688576, Policy Loss: -0.04879593104124069, Value Loss: 0.20626376569271088, Entropy Loss: 0.8066115379333496
Update: 5564, Loss: 0.031110815703868866, Policy Loss: -0.04330543801188469, Value Loss: 0.1682109385728836, Entropy Loss: 0.9689215421676636
Update: 5565, Loss: -0.009725840762257576, Policy Loss: -0.06219106912612915, Value Loss: 0.12419205904006958, Entropy Loss: 0.9630801677703857
Update: 5566, Loss: 0.0430082343518734, Policy Loss: -0.04257245734333992, Value Loss: 0.18780872225761414, Entropy Loss: 0.8323668241500854
Update: 5567, Loss: 0.02970053255558014, Policy Loss: -0.06306294351816177, Value Loss: 0.20457734167575836, Entropy Loss: 0.9525195956230164
Update: 5568, Loss: 0.07001492381095886, Policy Loss: -0.042233556509017944, Value Loss: 0.24183893203735352, Entropy Loss: 0.8670985698699951
Update: 5569, Loss: 0.02508478

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-1500.mp4
Update: 5604, Loss: 0.04479938745498657, Policy Loss: -0.034528397023677826, Value Loss: 0.1779802292585373, Entropy Loss: 0.9662330746650696
Update: 5605, Loss: 0.057347144931554794, Policy Loss: -0.01525186374783516, Value Loss: 0.16352255642414093, Entropy Loss: 0.9162265062332153
Update: 5606, Loss: 0.03896605595946312, Policy Loss: -0.03842464089393616, Value Loss: 0.17496357858181, Entropy Loss: 1.009109377861023
Update: 5607, Loss: -0.00916019082069397, Policy Loss: -0.06871302425861359, Value Loss: 0.13966843485832214, Entropy Loss: 1.0281383991241455
Update: 5608, Loss: 0.040107157081365585, Policy Loss: -0.028781305998563766, Value Loss: 0.15484127402305603, Entropy Loss: 0.8532174825668335
Update: 5609, Loss: 0.024790074676275253, Policy Loss: -0.043670304119586945, Value Loss: 0.15423841774463654, Entropy Loss: 0.8658831119537354
Update: 5610, Loss: 0.06068956

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-1500.mp4
Update: 5611, Loss: -0.004174687899649143, Policy Loss: -0.0708690732717514, Value Loss: 0.15506738424301147, Entropy Loss: 1.0839307308197021
Update: 5612, Loss: 0.039464469999074936, Policy Loss: -0.03922436758875847, Value Loss: 0.17188149690628052, Entropy Loss: 0.7251911759376526
Update: 5613, Loss: 0.05092056468129158, Policy Loss: -0.021028678864240646, Value Loss: 0.1599859744310379, Entropy Loss: 0.8043743371963501
Update: 5614, Loss: 0.03616650775074959, Policy Loss: -0.027344856411218643, Value Loss: 0.1464405655860901, Entropy Loss: 0.9708918929100037
Update: 5615, Loss: 0.06650661677122116, Policy Loss: -0.03836720436811447, Value Loss: 0.22651927173137665, Entropy Loss: 0.838581383228302
Update: 5616, Loss: 0.0707274079322815, Policy Loss: -0.031029976904392242, Value Loss: 0.21791279315948486, Entropy Loss: 0.7199011445045471
Update: 5617, Loss: 0.073612362

Moviepy - Done !
Moviepy - video ready /home/ananya-acharya/Projects/rl_atari/videos/Pong/rl-video-episode-1500.mp4
Update: 5624, Loss: 0.022846907377243042, Policy Loss: -0.05159773677587509, Value Loss: 0.16705076396465302, Entropy Loss: 0.908073902130127
Update: 5625, Loss: -0.0008414657786488533, Policy Loss: -0.06595903635025024, Value Loss: 0.14827775955200195, Entropy Loss: 0.9021309018135071
Update: 5626, Loss: 0.033091865479946136, Policy Loss: -0.027885261923074722, Value Loss: 0.13998378813266754, Entropy Loss: 0.9014768600463867
Update: 5627, Loss: 0.06753508001565933, Policy Loss: -0.042685896158218384, Value Loss: 0.23839956521987915, Entropy Loss: 0.8978806734085083
Update: 5628, Loss: 0.029971487820148468, Policy Loss: -0.050362445414066315, Value Loss: 0.17559611797332764, Entropy Loss: 0.7464125752449036
Update: 5629, Loss: 0.07940559089183807, Policy Loss: 0.0023373886942863464, Value Loss: 0.17211896181106567, Entropy Loss: 0.8991276025772095
Update: 5630, Loss: 0.0

In [41]:
envs.close()